In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras import models
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
# Нормалізація значень пікселів до діапазону [0, 1]
train_images_resized = train_images/255
test_images_resized = test_images/255

# Повторення одного каналу в три
train_images_resized = tf.image.grayscale_to_rgb(tf.expand_dims(train_images_resized, axis=-1))
test_images_resized = tf.image.grayscale_to_rgb(tf.expand_dims(test_images_resized, axis=-1))

# Збільшення розміру зображень до (32, 32)
item_size = 32,32
train_images_resized = tf.image.resize(train_images_resized, (item_size))
test_images_resized = tf.image.resize(test_images_resized, (item_size))

train_labels_onehot = to_categorical(train_labels)
test_labels_onehot = to_categorical(test_labels)

In [4]:
# Побудова моделі
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(item_size[0],item_size[1], 3))
conv_base.trainable = False

model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dense(10, activation="sigmoid"),
])

In [5]:
model.load_weights('saved_model_weights.h5')

In [6]:
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    metrics=["acc"]
)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [7]:
history = model.fit(train_images_resized, train_labels_onehot, epochs=150, batch_size=128, validation_split=0.2,
    callbacks=[early_stopping])

Epoch 1/150
375/375 [==============================] - 282s 750ms/step - loss: 0.0582 - acc: 0.8869 - val_loss: 0.0598 - val_acc: 0.8832
Epoch 2/150
375/375 [==============================] - 269s 719ms/step - loss: 0.0581 - acc: 0.8875 - val_loss: 0.0598 - val_acc: 0.8838
Epoch 3/150
375/375 [==============================] - 262s 698ms/step - loss: 0.0580 - acc: 0.8878 - val_loss: 0.0599 - val_acc: 0.8837
Epoch 4/150
375/375 [==============================] - 253s 676ms/step - loss: 0.0580 - acc: 0.8873 - val_loss: 0.0599 - val_acc: 0.8834
Epoch 5/150
375/375 [==============================] - 273s 727ms/step - loss: 0.0579 - acc: 0.8875 - val_loss: 0.0600 - val_acc: 0.8829


In [8]:
model.save_weights('saved_model_weights.h5')

Здається що модель погано адаптується під зображення в одному кольорі, можливо зі збільшенням розміру та ретельнішим підбором гіперпараметрів можна поліпшити передбачення, але це збільшить час навчання.Модель з повнозв'язними шарами з попередньої домашньої роботи виконує це завдання краще при меншому часі на навчання.Інша згорткова мережа з DS_HW10 виявилась видшою в навчанні що дало змогу заощадити час та підібрати кращі параметри для моделі.Згорткові мережі дають кращий результат передбачення ніж повнозв'язні, найкращий резуьтат з попередньої роботи 89% на валідаційних даних, згорткова мережа з першої частини цього завдання має 92%

In [10]:
test_loss, test_acc = model.evaluate(test_images_resized,  test_labels_onehot, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 47s - loss: 0.0671 - acc: 0.8661 - 47s/epoch - 150ms/step

Test accuracy: 0.866100013256073
